### SEARCH HGBOOST

In [54]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,StratifiedKFold

In [55]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [56]:
data = pd.read_csv('HR.csv')

target = 'left'
class_names = ['stayed', 'left']
features = [c for c in data if c != target]
print(features)

X, y = data[features], data[target]

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 7 columns):
last_evaluation          14999 non-null float64
number_project           14999 non-null int64
average_montly_hours     14999 non-null int64
time_spend_company       14999 non-null int64
Work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
dtypes: float64(1), int64(6)
memory usage: 820.3 KB


In [58]:
X.tail()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
14994,0.57,2,151,3,0,0
14995,0.48,2,160,3,0,0
14996,0.53,2,143,3,0,0
14997,0.96,6,280,4,0,0
14998,0.52,2,158,3,0,0


In [59]:
data["left"].value_counts()

0    9821
1    5178
Name: left, dtype: int64

### Обучение

In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
model = xgb.XGBClassifier(seed=42)

In [ ]:
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [43]:
#Choose some parameter combinations to try
parameters = {'max_depth': [2, 3, 5, 6, 12, 25, 50],
    'n_estimators' : [5, 10, 20, 50, 100, 200, 500, 1000],
    'learning_rate' : [0.05, 0.08, 0.09, 0.1, 0.11, 0.12, 0.15],
    'min_child_weight': [1, 2, 3, 5]
}

# Run the grid search
grid_obj_xgb = GridSearchCV(model, parameters, scoring='accuracy', 
                            cv = 3, n_jobs= 2, verbose = 1)
grid_obj_xgb = grid_obj_xgb.fit(X, y)

# Set the clf to the best combination of parameters
model_best = grid_obj_xgb.best_estimator_
print(grid_obj_xgb.best_params_, grid_obj_xgb.best_score_)

Fitting 3 folds for each of 1568 candidates, totalling 4704 fits


KeyboardInterrupt: 

In [53]:
model_try = xgb.XGBClassifier(seed = 42, n_estimators=56, max_depth=7, learning_rate=0.1, min_child_weight=1)
score = np.mean(cross_val_score(model_try, X, y, scoring='accuracy', cv=3))

print(score)

0.781654877642


In [75]:
model_try = xgb.XGBClassifier(n_estimators=100, max_depth=7, learning_rate=0.5)

In [76]:
print(cross_val_score(model_try, X, y, verbose = True, scoring='accuracy').mean())

0.740584663599


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.6s finished


In [108]:
def func(x):
    model = xgb.XGBClassifier(max_depth=int(x[0]), learning_rate=x[1], n_estimators=int(x[2]), min_child_weight=x[3])
    score = cross_val_score(model, X, y, scoring='accuracy', cv=3).mean()
    return score

In [109]:
func([10, 0.1, 100, 1])

0.77118714409548572

In [110]:
from scipy import optimize

In [114]:
opt = optimize.minimize(func, (10, 0.1, 1000, 1), method='SLSQP', bounds=((2,100),(0.9, 1.1), (5, 2000), (1, 5)))

In [115]:
opt.fun

0.7211826765353071

In [87]:
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2

In [88]:
bnds = ((0, None), (0, None))

In [90]:
res = optimize.minimize(fun, (2, 0), method='SLSQP', bounds=bnds)